# Домашнее задание

Используя информацию по мидиям Петербурга и Магадана, которые представлены в виде двух массивов petersburg и magadan в разделе ANOVA тест, проверьте данные на нормальность и на наличие корреляции.\
Для этого объедините 2 массива в DataFrame. Обоснуйте выбор теста на корреляцию. Сделайте вывод по гипотезе.
 
 
Также ответьте на вопрос:
Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.

In [1]:
import pandas as pd
import numpy as np

In [2]:
petersburg = [0.0974, 0.1352, 0.0817, 0.1016, 0.0968, 0.1064, 0.105]
magadan = [0.1033, 0.0915, 0.0781, 0.0685, 0.0677, 0.0697, 0.0764, 0.0689]

Массивы данных разной длины. Для создания dataframe исходные данные должны быть одной длины.
Пока добавлю в petersburg nan.

In [3]:
new_petersburg = petersburg.copy()
new_petersburg.append(np.nan)

In [4]:
df = pd.DataFrame({'petersburg': new_petersburg, 'magadan': magadan})
df

,petersburg,magadan
0,0.0974,0.1033
1,0.1352,0.0915
2,0.0817,0.0781
3,0.1016,0.0685
4,0.0968,0.0677
5,0.1064,0.0697
6,0.1050,0.0764
7,NaN,0.0689


NaN в признаке petersburg нужно чем-то заполнить. 

По логике размеры мидий, это размеры из материального мира из природы. 

Они должны быть подчинены законам нормального распределения, тогда пропуск можно заполнить средней.

Но предварительно сделаю тест на нормальность исходного списка petersburg

In [5]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

# Уровень значимости
alpha = 0.05

Использую Тест Шапиро-Уилка

In [6]:
from scipy.stats import shapiro

In [7]:
_, p = shapiro(petersburg)
print(f'p={p:.3f}')

# Интерпретация 
if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.242
Данные распределены нормально


Заполняю пропуск в данных petersburg средним значением

In [8]:
df['petersburg'].fillna(df['petersburg'].mean(), inplace=True)

In [9]:
df

,petersburg,magadan
0,0.097400,0.1033
1,0.135200,0.0915
2,0.081700,0.0781
3,0.101600,0.0685
4,0.096800,0.0677
5,0.106400,0.0697
6,0.105000,0.0764
7,0.103443,0.0689


# Проверка всех данных на нормальность

In [10]:
H0 = 'Данные распределены нормально'
Ha = 'Данные не распределены нормально (мы отвергаем H0)'

Установим уровень значимости

In [11]:
alpha = 0.05

## Тест Шапиро-Уилка

In [12]:
_, p = shapiro(df)
print(f'p={p:.3f}')

# Интерпретация 
if p > alpha:
	print(H0)
else:
	print(Ha)

p=0.100
Данные распределены нормально


Тест Д’Агостино не буду использовать т.к. для его применения в выборке должно быть > 20 значений

Т.к. нулевая гипотеза не была отвергнута и мы считаем что данные распределены нормально, то корреляцию определяем по пирсону

# Корреляция Пирсона 

In [13]:
from scipy.stats import pearsonr

In [14]:
corr, p = pearsonr(df['petersburg'], df['magadan'])

In [15]:
print(corr, p)

0.24455753997284532 0.5594098062056255


In [17]:
# Стандартный способ определения корреляции
df.corr()

,petersburg,magadan
petersburg,1.000000,0.244558
magadan,0.244558,1.000000


In [87]:
if p > alpha:
    print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.")
else:
    print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу об отсутсвии зависимости между переменными.")

0.5594098062056255 > 0.05. Мы не можем отвергнуть нулевую гипотезу об отсутсвии зависимости между переменными.


---

#### Какова разница в среднем размере мидии в зависимости от города-производителя. Обоснуйте выбор теста. Сделайте вывод по гипотезе.

Ранее установлено что данные распределены нормально.

Зависиманя переменная - средний размер мидий. Количественная.

Сравниваются две группы, два города.

Объектов в выборке < 30.

Выбираю параметрический Независимый T-тест т.к. группы (замеры мидий) поисходят из разных совокупностей(городов)

In [91]:
from scipy.stats import ttest_ind

In [92]:
# Формирую гипотезы
H0 = 'Нет значимой разницы между средними размерами мидий в двух разных городах.'
Ha = 'Есть значимая разница между средними размерами мидий в двух разных городах.'

In [95]:
def t_test(df):
    print('\n' + "*** Результаты независимого T-теста ***")
    test_results = ttest_ind(df['petersburg'], df['magadan'], equal_var=True)

    p = round(test_results[1], 2)

    if p > alpha:
        print(f"{p} > {alpha}. Мы не можем отвергнуть нулевую гипотезу. {H0}")
    else:
        print(f"{p} <= {alpha}. Мы отвергаем нулевую гипотезу. {Ha}")

t_test(df)


*** Результаты независимого T-теста ***
0.0 <= 0.05. Мы отвергаем нулевую гипотезу. Есть значимая разница между средними размерами мидий в двух разных городах.
